<a href="https://colab.research.google.com/github/Approach-Analytics/Emotion-Classifier/blob/main/Train_Fear_Distil_TF_May20%2C2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Distillbert Fear Emotoin Classification - Optimized for T3

The issues that we are going to need to address are:


1.   What are compute requirements...
2.   How does the whole tokenization / embedding thing work...
3.   What training set(s) to use?
4.   What model to use that doesn't mess with our inference?


Resources:
https://www.tensorflow.org/text/tutorials/classify_text_with_bert



#Import libraries

In [1]:
!pip install tensorflow
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification

In [3]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import pickle

In [4]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


In [5]:
#Dec 7, 2022: Not sure what this piece of code is used for...

import warnings
warnings.filterwarnings('ignore')

In [6]:
#Setting column options

pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 100)

#Supressing the scientific notation

pd.set_option('display.float_format', lambda x: '%.3f' % x)

#Import the dataset

In [7]:
# Load the Drive helper and mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#Older dataset

#path = "/content/drive/MyDrive/Sean/Emoclass/Emotion datasets/Fear_Twit_Kaggle_110K_May9,2023.csv"
#df = pd.read_csv(path)

In [9]:
#Trying new dataset

path = '/content/drive/MyDrive/Sean/Emoclass/Emotion datasets/Fear_Gut55K_2.8mil_June2.csv'
df = pd.read_csv(path)

In [10]:
df.shape

(3241651, 3)

In [11]:
df.head()

,text,emotion1,emotion2
0,endent\nefforts\nto\ngain\na\nlivelihood\nhe\n...,concerned,concern
1,o\ntwo\ndollars\nand\ncents\nalmost\nthree\ndo...,afraid,fear
2,siderably\nrelieved\nwhin\ncan\ni\nget\nit\ni\...,afraid,fear
3,\nhave\nyou\nthank\nyou\nsam\nsaid\nhoffman\nw...,afraid,fear
4,n\nhim\nand\nmoreover\nhad\na\nhundred\ndollar...,anxiety,anxiety


In [12]:
# Create a 50% sample of the DataFrame

df = df.sample(frac=0.3, random_state=42)

In [13]:
df.shape

(972495, 3)

In [14]:
#Relabelling a column...

df.rename(columns={"emotion2": "label"},inplace =True)

#Import the emotion axis

In [15]:
#Defining the emotion axis - note the use of the ordered dict here...

from collections import OrderedDict

emo_axis = OrderedDict([
    ("dread", ['panic','dread','horror','horrified','horrifying','terror','terrified','terrifying']),
    ("fear", ["fear", "fearful", "fright","frightening","afraid","frightful","frightfully","frightened",'scared','scary','scare']),
    ("anxiety", ["anxious","anxiety",'angst','anxiousness']),
    ("worry", ["worry", "worried","worrying","worries"]),
    ("concern", ["concern", "concerning","concerned","concerns"]),
    ("calm", ["calm","peaceful","serene","serenity","untroubled", "content", "contented", "composed", "tranquil"])
])


In [16]:
# Assign numerical labels to emotion categories in reversed order
labels = list(emo_axis.keys())
label_mapping = {label: i for i, label in enumerate(reversed(labels))}


In [17]:
#check the label mapping

label_mapping

{'calm': 0, 'concern': 1, 'worry': 2, 'anxiety': 3, 'fear': 4, 'dread': 5}

#Remove key emotion words

In [18]:
#check the right label is used...

#df['emotion1'].value_counts()

In [19]:
#Generate all the unique emotion words that then get replaced...

a = df['emotion1'].unique().tolist()

#The list a is our list of variable responses from the dataset...
keyword = "emotions"
words = a
for j in words:
  df['text'] = df['text'].str.replace(j,keyword)

##Splitting into train, validate and test

In [20]:
#This outputs 3 different dataframes... originally 0.6 and 0.8

train, validate, test = np.split(df.sample(frac=1, random_state=42),
                       [int(.8*len(df)), int(.9*len(df))])

##Preparing the Labels

In [21]:
#Converting the pandas dataframe into a list of labels...
#We may consider puting this into a function...

trainlabel=train['label'].tolist()
vallabel=validate['label'].tolist()
testlabel=test['label'].tolist()

In [22]:
#Test the label mapping again - dyslexic after all...

label_mapping

{'calm': 0, 'concern': 1, 'worry': 2, 'anxiety': 3, 'fear': 4, 'dread': 5}

In [23]:
# Convert the train, validation, and test labels using label_mapping
train_labels = [label_mapping[label] for label in trainlabel]
val_labels = [label_mapping[label] for label in vallabel]
test_labels = [label_mapping[label] for label in testlabel]


In [24]:
#Convert labels to Tensors (int32)

train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_labels, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

#Training and Tokenizing Tweets

---



In [25]:
#Converting the different pandas dataframes into a list of text fields...
#Choice of variables: truncated_text, text_minus1, 'filtered_text',''filtered_text_minus1'

traintext=train['text'].tolist()
valtext=validate['text'].tolist()
testtext=test['text'].tolist()

##Load the Tokenizer

In [26]:
#Load the tokenizer...

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [27]:
#Tokenize the data...

train_encodings = tokenizer(traintext, truncation=True, padding=True)
val_encodings = tokenizer(valtext, truncation=True, padding=True)
test_encodings = tokenizer(testtext, truncation=True, padding=True)

In [28]:
#Create tensoflow datasets

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))


# Load compile and Train the Model

In [29]:
#Load the model

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [30]:
#Compile the model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [31]:
#Fit the model...

model.fit(train_dataset.shuffle(1000).batch(50), epochs=3,validation_data=val_dataset.batch(50))

Epoch 1/3
 2140/15560 [===>..........................] - ETA: 5:12:39 - loss: 0.7178 - accuracy: 0.7319

KeyboardInterrupt: ignored

#Evaluate the model

In [ ]:
model.evaluate(test_dataset.batch(16))

In [ ]:
#Make predictions on test data

predictions = model.predict(test_dataset.batch(16))

In [ ]:
#Convert prediction score to predicted classes

predicted_classes = tf.argmax(predictions.logits, axis=-1)

predicted_classes = tf.cast(tf.argmax(predictions.logits, axis=-1), dtype=tf.int32)

#Calculate absolute difference

abs_diffs = tf.abs(predicted_classes - test_labels)

In [ ]:
#Create histogram of absolute differences

import matplotlib.pyplot as plt

plt.hist(abs_diffs.numpy(), bins=11)
plt.xlabel('Absolute difference')
plt.ylabel('Frequency')
plt.title('Distribution of absolute differences between predicted and true classes')
plt.show()


#Save the tokenizer and the model

In [ ]:
#path = /content/drive/MyDrive/Sean/Emoclass

# Save the model
#model.save_pretrained('/content/drive/MyDrive/Sean/Emoclass/Model_Fear_Twitter_100K_Distill_May20,2023')

# Save the tokenizer
#tokenizer.save_pretrained('/content/drive/MyDrive/Sean/Emoclass/Distill_Tokenizer_May20')
